In [2]:
class data:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
args = data()
args.filename = "paultan.csv"
args.stemmer = "lemma"
args.dictionary = "automotive"
args.model = "dtm"

In [3]:
# -*- coding: utf-8 -*-
import numpy as np
import lda
import lda.datasets
import csv
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from gensim import corpora, models

from stemming.porter2 import stem
from nltk.stem import *
import unicodecsv
import re
import os
import pyLDAvis.gensim
import gensim

_digits = re.compile('\d')
def contains_digits(d):
    return bool(_digits.search(d))

import enchant
d = enchant.Dict("en_US")
# Or using the /usr/share/dict/british-english word list
with open(args.dictionary + "-english") as word_file:
  english_words = set(word.strip().lower() for word in word_file)
  print(english_words)
  def is_english_word(word):
    return word.lower() in english_words

def process_tokens(tokens,stemmer):
  tokens = [i for i in tokens if not i in en_stop and not contains_digits(i) and is_english_word(i)]
  if stemmer == 'porter':
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(i) for i in tokens]
  elif stemmer == 'porter2':
    tokens = [stem(i) for i in tokens]
  elif stemmer == 'lemma':
    lemmatiser = WordNetLemmatizer()
    tokens = [lemmatiser.lemmatize(i) for i in tokens]
  return tokens


if args.filename == "sample":
  X = lda.datasets.load_reuters()
  dictionary = lda.datasets.load_reuters_vocab()
  titles = lda.datasets.load_reuters_titles()
else:
  dataset_filepath = os.path.join(dir, 'datasets/' + args.filename)
  f = open(dataset_filepath)
  reader = unicodecsv.reader(f, encoding='utf-8')
  # csv_length = sum(1 for row in reader)
  # f.seek(0) #reset reader position
  identifiers = reader.next()
  contents_idx = identifiers.index("contents")
  title_idx = identifiers.index("title")

  contents = [ row[contents_idx] for row in reader if row[contents_idx] ]

  f.seek(0)
  reader.next()
  titles = [ row[title_idx] for row in reader if row[contents_idx] ]



  texts = list()
  tokenizer = RegexpTokenizer(r'\w+')
  en_stop = get_stop_words('en')
  for idx,i in enumerate(contents):
#     if not idx % 10:
#       print "INFO: Tokenizing articles <{}> ".format(idx)
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)
    texts.append(process_tokens(tokens, args.stemmer))
    # print idx
    # add tokens to list

  print "[DEBUG] Length of Texts : {}".format(len(texts))
  dictionary = corpora.Dictionary(texts)
  corpus = [dictionary.doc2bow(text) for text in texts]


set(['changfeng motor', 'hi-test', 'magnetic', 'alternatives', 'dynasty', 'obstruction', 'bonneville salt flats', 'query', 'loyalty', 'adjustment', 'integrity', 'gonow', 'synthetic', 'saleswoman', 'disposal', 'technique', 'all-wheel drive', 'corrosive', 'jalopy', 'jianghuai', 'safety', 'infant seats', 'gravel', 'roadway', 'protective', 'lessons', 'advantage', 'dynamic', 'activation', 'aston martin', 'faw', 'mph', 'liability', 'balancing', 'drinking', 'lubrication', 'courageous', 'condition', 'tickets', 'retrial', 'transmission', 'race', 'jiangnan', 'recall', 'quick', 'inspection', 'accumulation', 'triumph', 'revolution', 'occurrence', 'corvette', 'four door', 'horn', 'tires', 'cost', 'design', 'enhancement', 'rigorous', 'runner-up', 'wings', 'hongqi', 'wreck', 'fine', 'specification', 'overload', 'sequential', 'optional', 'roewe', 'international', 'changhe', 'appeal', 'racing', 'ignition', 'impact', 'zhongyu', 'jiangling', 'component', 'chassis', 'lubricant', 'hwanghai', 'pedals', 'mac

AttributeError: 'builtin_function_or_method' object has no attribute 'endswith'

In [14]:
pyLDAvis.enable_notebook()
  model_filename = os.path.join(dir, 'models/' + args.filename.split('.')[0] + "_" + args.model + '.model')
try:
  ldamodel = models.LdaModel.load(model_filename)
except IOError:
  if(args.model == "lda"):
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes=20)
  elif(args.model == "dtm"):
    my_timeslices = [500,500,500,500,500,346]
    ldamodel = gensim.models.wrappers.DtmModel('/Users/Hii/Projects/news_scraper/dtm-darwin64', corpus, my_timeslices, num_topics=20, id2word=dictionary,initialize_lda=True)
  else:
    raise ValueError('Unknown Model Type')
  ldamodel.save(model_filename)


# ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes=20)
# pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)



In [17]:
  for idx, topic in enumerate(ldamodel.show_topics(topics=10, topn=10, log=False, formatted=False)):
    print "Topic #" + str(idx) + " :",
    for word in topic:
      print word[1],
    print

Topic #0 : year safety increase price car impact point vehicle test cost
Topic #1 : volkswagen perodua year recall scandal brand vehicle automaker model new
Topic #2 : audi drive cockpit new navigation lot comfort map showroom information
Topic #3 : car porsche speed power engine drive weight wheel performance chassis
Topic #4 : design ferrari fiat bentley jeep car year feature new dashboard
Topic #5 : engine speed new transmission automatic torque manual cylinder model mitsubishi
Topic #6 : toyota vehicle new accessory year car part technical passenger fuel
Topic #7 : engine fuel new power efficiency performance oil consumption cylinder technology
Topic #8 : new car model sedan year design debut feature make engine
Topic #9 : control feature new speed display drive safety leather automatic model
Topic #10 : year safety increase price car impact point vehicle test cost
Topic #11 : volkswagen perodua year recall scandal brand vehicle automaker model new
Topic #12 : audi drive cockpit ne